In [1]:
%load_ext sql
%sql --section duck
import epyfun
sqlite_file = epyfun.get_latest_file("./data/")

Loading configurations from C:\OneDrive\All\strom\pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaycon,False
dsn_filename,./connections.ini


Connecting to 'duck'

In [2]:
%%sql

            SELECT 
                year, month, day, hour,
                "1_cd" AS nd,
                "2_cd" + "3_cd" AS wd,
                "2_cd" AS nt,
                "3_cd" AS ht,
                POW(("1_weight" + "2_weight" + "3_weight") / 3, 2) AS weight
            FROM (
                WITH cte AS (
                    SELECT     
                    EXTRACT(YEAR FROM minute) AS year,    
                    EXTRACT(MONTH FROM minute) AS month,
                    EXTRACT(DAY FROM minute) AS day,
                    EXTRACT(HOUR FROM minute) AS hour,
                    1 / FIRST_VALUE(minutes IGNORE NULLS) OVER(
                        PARTITION BY meterid 
                        ORDER BY minute 
                        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
                    ) AS weight,
                    * 
                    FROM strom_minute
                    WHERE 
                        (minute >= '2020-12-01' AND minute <= '2021-05-25') 
                        OR 
                        (minute >= '2022-12-01')
                )
                PIVOT_WIDER cte
                ON meterid
                USING 
                    SUM(cm) AS cd,
                    --AVG(cm* 24.0 * 60.0)  AS cd, 
                    AVG(weight) AS weight
                GROUP BY year, month, day, hour
            )
            ;


,year,month,day,hour,nd,wd,nt,ht,weight
0,2020,12,1,1,0.273349,1.231004,0.820046,0.410959,5.196749e-06
1,2020,12,1,21,0.545455,1.367372,0.821918,0.545455,2.072410e-05
2,2020,12,2,1,0.000000,0.669643,0.401786,0.267857,4.989895e-06
3,2020,12,2,21,0.289157,1.084337,0.289157,0.795181,1.451589e-06
4,2020,12,3,4,0.160000,1.120000,0.480000,0.640000,7.111111e-06
...,...,...,...,...,...,...,...,...,...
17550,2023,9,16,17,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17551,2023,9,17,2,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17552,2023,9,17,10,0.407793,0.170885,0.112629,0.058256,4.189858e-09
17553,2023,9,17,18,0.407793,0.170885,0.112629,0.058256,4.189858e-09


In [1]:
%%sql
SELECT *,
            1 / FIRST_VALUE(minutes IGNORE NULLS) OVER(
                PARTITION BY meterid 
                ORDER BY minute 
                ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING 
            ) AS weight,

FROM strom_minute
WHERE meterid = 1
--LIMIT 50 OFFSET 100089
;

UsageError: Cell magic `%%sql` not found.


In [3]:
%load_ext sql
%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%%sql

CREATE TABLE services AS
    FROM 'https://blobs.duckdb.org/nl-railway/services-2023.csv.gz';

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "services" already exists!
[SQL: CREATE TABLE services AS
    FROM 'https://blobs.duckdb.org/nl-railway/services-2023.csv.gz';]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [8]:
%%sql

SELECT format('{:,}', count(*)) AS num_services
FROM services;

,num_services
0,"21,239,393"


In [13]:
%%sql

DESCRIBE TABLE services;

,Success


In [1]:
import scipy.optimize as opt


# Define the function for the present value of an annuity
def present_value_annuity(r, X, PMT, n):
    return PMT * (1 - (1 + r) ** -n) / r - X


# Initial values
X = 37840  # Example initial amount
PMT = 242.57
n = 300

# Use a solver to find the interest rate
r = opt.newton(present_value_annuity, x0=0.05 / 12, args=(X, PMT, n))
annual_interest_rate = r * 12 * 100  # Convert monthly rate to annual percentage rate

print(f"The annual interest rate is approximately {annual_interest_rate:.7f}%")

The annual interest rate is approximately 5.9466018%


In [2]:
def calculate_monthly_payment(X, N, i):
    # Convert annual interest rate to monthly interest rate
    r = i / 12

    # Calculate the monthly payment using the formula
    PMT = X * (r * (1 + r) ** N) / ((1 + r) ** N - 1)

    return PMT


# Example values
X = 37840  # Loan amount
N = 15 * 12  # Total periods (months)
i = 0.04  # Annual interest rate (5%)

# Calculate monthly payment
monthly_payment = calculate_monthly_payment(X, N, i)
print(f"The monthly payment is approximately ${monthly_payment:.2f}")

The monthly payment is approximately $279.90
